In [ ]:
import os
from dotenv import load_dotenv
load_dotenv()
os.environ.get('SENDGRID_ALT_KEY')

In [7]:
import sendgrid
import os
from dotenv import load_dotenv
load_dotenv()
sg = sendgrid.SendGridAPIClient(os.environ.get('SENDGRID_API_KEY'))


In [ ]:
# test SendGrid mailing list operations
import sendgrid
import os
from dotenv import load_dotenv
load_dotenv()
sg = sendgrid.SendGridAPIClient(os.environ.get('SENDGRID_API_KEY'))
params = {'page': 1, 'page_size': 1}
response = sg.client.contactdb.recipients.get(query_params=params)
print(response.status_code)
print(response.body)
#print(response.headers)


In [ ]:
# Sendgrid retrieve recipients
import os
from sendgrid import SendGridAPIClient


sg = SendGridAPIClient(os.environ.get('SENDGRID_API_KEY'))
response = sg.client.marketing.contacts.exports.post()
id = response.body.decode("utf-8")
id

In [10]:
from sendgrid import SendGridAPIClient
import os
import json
import time
import pandas as pd
import requests
from io import StringIO

sg = SendGridAPIClient(os.environ.get('SENDGRID_API_KEY'))
response = sg.client.marketing.contacts.exports._(json.loads(id)['id']).get()
time.sleep(10)
urls = response.body.decode("utf-8")
download = json.loads(urls)["urls"][0]
req = requests.get(download)
df = pd.read_csv(StringIO(req.text))
sg_emails = df['EMAIL'].tolist()

parse index.xml to get latest feed item

In [13]:
import feedparser

feed = feedparser.parse('../docs/index.xml')
newsletter = feed.entries[0]
print(newsletter.title)
print(newsletter.description)

ADCN: Open Biomechanics Research, U.S. Sport Policy, News
<p>The Athletes Data Community Newsletter (ADCN) is produced by <a href="https://newsletter.bradstenger.com/">Brad Stenger</a>, a researcher studying applied privacy technologies in the context of athletes’ data at the University of Vermont. The newsletter covers the athletes’ health and performance data pipeline, from collection to management to reporting, and the factors affecting athletes’ data. Thanks for reading!</p>
<section class="level3" id="university-research-in-open-biomechanics">
<h3 class="anchored">University Research in Open Biomechanics</h3>
<p>The KneeHub project just published <a href="https://asmedigitalcollection.asme.org/biomechanical/article/doi/10.1115/1.4070823/1229924/Deciphering-the-Art-in-Modeling-and-Simulation-of">an update</a> to a biomechanics models benchmarking <a href="https://pmc.ncbi.nlm.nih.gov/articles/PMC8086182/">study</a> first published in 2021. KneeHub was conceived as a way to determin

In [14]:
import re
from bs4 import BeautifulSoup
soup = BeautifulSoup(newsletter.description, 'html.parser')
sections = soup.find_all('section')
subject = '"subject": "' + newsletter.title + '"'
introduction = '"introduction":' + '[' + ",".join([ '{"p":"' + re.sub(r'"', r'\"', str(p))  + '"}' for p in reversed(sections[0].find_previous_siblings('p'))]) + ']'
#section = '"sections":' + '[' + ",".join([ '{"section":"' + re.sub(r'"', r'\"', str(s)) + '"}' for s in sections]) + ']'
section = []
for s in sections:
    heading = '"heading": "' + re.sub(r'"', r'\"', str(s.find('h3'))) + '"'
    description = '"description": "' + re.sub(r'"', r'\"', str(s.find('p'))) + '"'
    if s.find('ul'):
        paragraphs = '"paragraphs":' + '[' + ",".join([ '{"p":"' + re.sub(r'"', r'\"', str(p))  + '"}' for p in s.find('li').find_next_siblings('li')]) + ']'
    else:
        paragraphs = '"paragraphs":' + '[' + ",".join([ '{"p":"' + re.sub(r'"', r'\"', str(p))  + '"}' for p in s.find('p').find_next_siblings('p')]) + ']'
    sec = '{"section": {' + heading + ', ' + description + ', ' + paragraphs + '}}'
    section.append(sec)
h_sections = '"sections": [' + ",".join(section) + ']'
handlebar = '{' + subject + ', ' + introduction + ', ' + h_sections + '}'
print(handlebar)


{"subject": "ADCN: Open Biomechanics Research, U.S. Sport Policy, News", "introduction":[{"p":"<p>The Athletes Data Community Newsletter (ADCN) is produced by <a href=\"https://newsletter.bradstenger.com/\">Brad Stenger</a>, a researcher studying applied privacy technologies in the context of athletes’ data at the University of Vermont. The newsletter covers the athletes’ health and performance data pipeline, from collection to management to reporting, and the factors affecting athletes’ data. Thanks for reading!</p>"}], "sections": [{"section": {"heading": "<h3 class=\"anchored\">University Research in Open Biomechanics</h3>", "description": "<p>The KneeHub project just published <a href=\"https://asmedigitalcollection.asme.org/biomechanical/article/doi/10.1115/1.4070823/1229924/Deciphering-the-Art-in-Modeling-and-Simulation-of\">an update</a> to a biomechanics models benchmarking <a href=\"https://pmc.ncbi.nlm.nih.gov/articles/PMC8086182/\">study</a> first published in 2021. KneeHub 

In [ ]:
import sendgrid
import os
from sendgrid.helpers.mail import *
from dotenv import load_dotenv
load_dotenv()
emails = [] #..fill as needed
#emails = sg_emails
for email in emails:
	sg = sendgrid.SendGridAPIClient(os.environ.get('SENDGRID_API_KEY'))
	message = Mail(from_email=os.environ.get('SENDGRID_SENDER_EMAIL'), to_emails=email)
	message.dynamic_template_data = eval(handlebar)
	message.template_id = os.environ.get('SENDGRID_TEMPLATE_ID')
	response = sg.send(message)
	print(response.status_code, email)

202 jstenger@uvm.edu


In [ ]:
sg_emails